In [9]:
import open3d as o3d
import numpy as np
from typing import List
from skspatial.objects import Plane, Points
from skspatial.plotting import plot_3d
import matplotlib.pyplot as plt
import matplotlib
import time
import plotly.express as px
import pandas as pd
from PIL import Image

import os

In [ ]:
pcds: List[o3d.geometry.PointCloud] = []
# plys_dir = 'very_flat_plys/'
# plys_dir = 'road_long_towards_plys/'
plys_dir = '../../maps/flat_frame_separated/'

for fn in os.listdir(plys_dir):
    pcds.append(o3d.io.read_point_cloud(plys_dir + fn))

In [78]:
inliners = []
inliners_colors = []
outliners = []
outliners_err = []
inliers_pointcount = []
out10_pointcount = []
out15_pointcount = []

for i, pcd in enumerate(pcds):
    # if i == 10: break
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=10, std_ratio=1.0)

    plane_model, inliers = pcd.segment_plane(distance_threshold=0.1,
                                             ransac_n=3,
                                             num_iterations=1000)

    plane_model_err, inliers_err = pcd.segment_plane(distance_threshold=0.15,
                                             ransac_n=3,
                                             num_iterations=1000)

    inlier_cloud = pcd.select_by_index(inliers)
    outlier_cloud = pcd.select_by_index(inliers, invert=True)
    outlier_cloud_err = pcd.select_by_index(inliers_err, invert=True)
    outlier_cloud.paint_uniform_color([0.8, 0.5, 0])
    outlier_cloud_err.paint_uniform_color([1, 0, 0])

    # analysis
    # count how many points in outlier cloud
    inliers_pointcount.append(len(inlier_cloud.points))
    out10_pointcount.append(len(outlier_cloud.points))
    out15_pointcount.append(len(outlier_cloud_err.points))

    inliners.append(np.asarray(inlier_cloud.points))
    inliners_colors.append(np.asarray(inlier_cloud.colors))
    outliners.append(np.asarray(outlier_cloud.points))
    outliners_err.append(np.asarray(outlier_cloud_err.points))

    if i % 50 == 0:
        print(f'Finished {i+1}/{len(pcds)}')

inliners = np.concatenate(inliners)
inliners_colors = np.concatenate(inliners_colors)
outliners = np.concatenate(outliners)
outliners_err = np.concatenate(outliners_err)

Finished 1/256
Finished 51/256
Finished 101/256
Finished 151/256
Finished 201/256
Finished 251/256


In [79]:
print(out10_pointcount == inliers_pointcount)

False


In [86]:
def plot_outliers_count(outliers_pc_, inliers_pc_=[],idx_range_=()):
    # slicing
    outliers_pc_ = outliers_pc_ [idx_range_[0]:idx_range_[1]] if idx_range_ else outliers_pc_  
    inliers_pc_ = inliers_pc_ [idx_range_[0]:idx_range_[1]] if (idx_range_ and inliers_pc_)  else inliers_pc_ 

    _out_np = np.array([outliers_pc_]).transpose()
    _out_np = np.concatenate((_out_np,np.zeros((_out_np.shape[0],1))),axis=1)
    _out_df = pd.DataFrame(_out_np, columns=['point_count','category'])
    _out_df[['category']] = 'outliers'

    if inliers_pc_:
        _inliers_np = np.array([inliers_pc_]).transpose()
        _inliers_np = np.concatenate((_inliers_np,np.zeros((_inliers_np.shape[0],1))),axis=1)

        _indf = pd.DataFrame(_inliers_np, columns=['point_count','category'])
        _indf[['category']] = 'inliers'
        df = pd.concat((_indf, _out_df))
    else:
        df = _out_df

    fig = px.bar(df, y="point_count", color="category", barmode="group", text_auto=True)
    fig.show()

plot_outliers_count(out10_pointcount, inliers_pointcount)

In [83]:
35*256/376

23.829787234042552

In [92]:
_pc = pcds[100]
# o3d.visualization.draw_geometries([_pc])

pcd, _ = _pc.remove_statistical_outlier(nb_neighbors=10, std_ratio=1.0)

plane_model, inliers = pcd.segment_plane(distance_threshold=0.1,
                                            ransac_n=3,
                                            num_iterations=1000)

inliers_cloud = pcd.select_by_index(inliers)
outlier_cloud_10 = pcd.select_by_index(inliers, invert=True)


inliers_np = np.array([inliers_cloud.points]).transpose()
out10_np = np.array([out10_pointcount]).transpose()

print(inliers_np.shape)


inliers_np = np.concatenate((inliers_np,np.zeros((inliers_np.shape[0],1))),axis=1)
out10_np = np.concatenate((out10_np,np.zeros((out10_np.shape[0],1))),axis=1)


# inliers_np = np.concatenate(
#     (inliers_np, np.zeros((inliers_np.shape[0],1))),axis=1
#     )
# indf = pd.DataFrame(inliers_np, columns=['X','Y','Z','category'])

# out10_np = np.concatenate(
#     (out10_np, np.ones((out10_np.shape[0],1))),axis=1
#     )
# out10df = pd.DataFrame(out10_np, columns=['X','Y','Z','category'])

# indf[['category']] = 'inliers'
# out10df[['category']] = 'outliers'
# df = pd.concat((indf, out10df))

# fig = px.scatter_3d(df,x='X',y='Y',z='Z',color='category')
# fig.update_traces(marker=dict(size=3))
# fig.show()

(256, 1)


In [ ]:
inliers_np = np.array([inliers_pointcount]).transpose()
out10_np = np.array([out10_pointcount]).transpose()

inliers_np = np.concatenate((inliers_np,np.zeros((inliers_np.shape[0],1))),axis=1)
out10_np = np.concatenate((out10_np,np.zeros((out10_np.shape[0],1))),axis=1)

indf = pd.DataFrame(inliers_np, columns=['point_count','category'])
out10_df = pd.DataFrame(out10_np, columns=['point_count','category'])

indf[['category']] = 'inliers'
out10_df[['category']] = 'outliers'

df = pd.concat((indf, out10_df))

In [ ]:
inliners_pcd = o3d.geometry.PointCloud()
inliners_pcd.points = o3d.utility.Vector3dVector(inliners)
inliners_pcd.colors = o3d.utility.Vector3dVector(inliners_colors)

outliners_pcd = o3d.geometry.PointCloud()
outliners_pcd.points = o3d.utility.Vector3dVector(outliners)
outliners_pcd.paint_uniform_color([0.8, 0.5, 0.0])

outliners_err_pcd = o3d.geometry.PointCloud()
outliners_err_pcd.points = o3d.utility.Vector3dVector(outliners_err)
outliners_err_pcd.paint_uniform_color([1.0, 0.0, 0.0])

In [ ]:
o3d.visualization.draw_geometries([inliners_pcd, outliners_pcd, outliners_err_pcd])

In [ ]:
def flatten(pcd: o3d.geometry.PointCloud) -> o3d.geometry.PointCloud:
    pts = np.asarray(pcd.points).copy()
    colors = np.asarray(pcd.colors).copy()
    for pt in pts:
        pt[2] = 0.0

    new_pcd = o3d.geometry.PointCloud()
    new_pcd.points = o3d.utility.Vector3dVector(pts)
    new_pcd.colors = o3d.utility.Vector3dVector(colors)

    return new_pcd

flat_inliners = flatten(inliners_pcd)
flat_outliners = flatten(outliners_pcd)
flat_outliners_err = flatten(outliners_err_pcd)

In [ ]:
# flat_inliners_filt, _ = flat_inliners.remove_radius_outlier(50, 0.5)
flat_outliners_filt, _ = flat_outliners.remove_radius_outlier(50, 0.5)
flat_outliners_err_filt, _ = flat_outliners_err.remove_radius_outlier(50, 0.5)

In [ ]:
o3d.visualization.draw_geometries([flat_inliners, flat_outliners_filt, flat_outliners_err_filt])

In [ ]:
### Taking picture, not working really well

vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(flat_inliners)
vis.add_geometry(flat_outliners_filt)
vis.add_geometry(flat_outliners_err_filt)
vis.update_geometry(flat_inliners)
vis.update_geometry(flat_outliners_filt)
vis.update_geometry(flat_outliners_err_filt)
vis.poll_events()
vis.update_renderer()
vis.capture_screen_image('image.png')
vis.destroy_window()

In [ ]:
# R = flat_inliners.get_rotation_matrix_from_xyz((0.0, 0.0, -np.pi / 4))
# flat_inliners.rotate(R)
# flat_outliners_filt.rotate(R)
# flat_outliners_err_filt.rotate(R)

In [ ]:
pcd_combined = flat_inliners +  flat_outliners_filt + flat_outliners_err_filt
assert isinstance(pcd_combined, o3d.geometry.PointCloud)
if plys_dir == 'road_long_towards_plys/':
    # rotate to make it more horizontal for image
    R = flat_inliners.get_rotation_matrix_from_xyz((0.0, 0.0, -np.pi / 4))
    pcd_combined.rotate(R)
o3d.visualization.draw_geometries([pcd_combined])

In [ ]:
# inliners_points = np.asarray(flat_inliners.points).copy()
# inliners_colors = np.asarray(flat_inliners.colors).copy()
# outliners_points = np.asarray(flat_outliners_filt.points).copy()
# outliners_colors = np.asarray(flat_outliners_filt.colors).copy()
# outliners_err_points = np.asarray(flat_outliners_err_filt.points).copy()
# outliners_err_colors = np.asarray(flat_outliners_err_filt.colors).copy()

# all_points = np.concatenate([inliners_points, outliners_points, outliners_err_points])
# all_colors = np.concatenate([inliners_colors, outliners_colors, outliners_err_colors])
all_points = np.asarray(pcd_combined.points).copy()
all_colors = np.asarray(pcd_combined.colors).copy()

In [ ]:
max_dim = all_points.max(axis=0)
min_dim = all_points.min(axis=0)
span = (max_dim - min_dim)[:2]

shifted_points = all_points - min_dim

print(span)

In [ ]:
img_scale = 10
img_size = tuple((img_scale * span).astype('int') + 1)
print(img_size)
img = Image.new('RGB', img_size, color=(100, 100, 100))
pixels = img.load()

In [ ]:
for i, (point, color) in enumerate(zip(shifted_points, all_colors)):
    point = tuple((img_scale * point[:2]).astype('int'))
    color = tuple((256 * color).astype('int'))
    pixels[point] = color

    if i % 100000 == 0:
        print(f'{i}/{len(shifted_points)}')
print('Finished')

In [ ]:
img.show()

In [ ]:
img.save('map.png')

In [ ]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(flat_outliners_err_filt.cluster_dbscan(eps=0.1, min_points=50, print_progress=True))
print(labels.max())

In [ ]:
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
flat_outliners_err_filt.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([flat_outliners_err_filt])